In [1]:
# Initial setup:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
%load_ext autoreload
%autoreload 1

#For fancy plots
import matplotlib
from matplotlib import pyplot as plt
matplotlib.rcParams['mathtext.fontset'] = 'cm'
matplotlib.rcParams['font.family'] = 'STIXGeneral'
from matplotlib.ticker import FormatStrFormatter
import matplotlib.patches as mpatches
from matplotlib.colors import ListedColormap

#Import standard libraries
import numpy as np
import time
import seaborn as sns
import pandas as pd
from itertools import product

from scipy.interpolate import interp1d
from scipy.integrate import simps as simps
from scipy.integrate import cumtrapz as cumtrapz
from scipy.integrate import simps, solve_ivp
from scipy.constants import c as c_light #meters/seconds
from scipy.stats.distributions import chi2 as chi2_dist

c_light_km = c_light/1000 #km/s

import getdist
getdist.chains.print_load_details = False

# import repo functions:
import sys, os
import os
import git
path_git = git.Repo('.', search_parent_directories=True).working_tree_dir
path_data = os.path.join(os.path.dirname(path_git), 'GILA-output', 'paper')
path_figures = os.path.join(path_git, 'notebooks', 'figures')

os.chdir(os.path.join(path_git, 'fr_mcmc', 'utils'))
from supernovae import aparent_magnitude_th, chi2_supernovae
from constants import OMEGA_R_0, LAMBDA, L, KAPPA

In [2]:

def F_H(H, params, model):     
    lamb, L, beta, L_bar, r, s = params # L and L_bar have to be in units of H0^{-1}
    if model == 'GILA':
        lamb = 0; p = 3; q = 1
        FH = H**2 - beta * H**(2*r) * L_bar**(2*(r-1)) * np.exp(-beta*(L_bar*H)**(2*s)) #\
                  #+ lamb * H**(2*p) * L**(2*(p-1))     * np.exp(lamb*(L*H)**(2*q))

    elif model == 'BETA':
        lamb = 0; p = 1; q = 2; r = 1
        FH = H**2 - beta * H**2                        * np.exp(-beta*(L_bar*H)**(2*s)) #\
                  #+ lamb * H**(2*p) * L**(2*(p-1))     * np.exp(lamb*(L*H)**(2*q))

    return FH


def F_H_prime(H, params, model):
    lamb, L, beta, L_bar, r, s = params # L and L_bar have to be in units of H0^{-1}
   
    if model == 'GILA':
        lamb = 0; p = 3; q = 1
        aux = beta * np.exp(-beta*(L_bar*H)**(2*s)) * (L_bar*H)**(2*(r-1)) * (-r + s * beta * (L_bar*H)**(2*s)) #+\
              #lamb * np.exp(lamb*(L*H)**(2*q))      * (L*H)**(2*(p-1))     * (p  + q * lamb * (L*H)**(2*q))

    elif model == 'BETA':
        lamb = 0; p = 1; q = 2; r = 1
        aux = beta * np.exp(-beta*(L_bar*H)**(2*s))                        * (-1 + s * beta * (L_bar*H)**(2*s)) #+\
              #lamb * np.exp(lamb*(L*H)**(2*q))      * (L*H)**(2*(p-1)) * (p  + q * lamb * (L*H)**(2*q))

    FH_prime = 2 * H * (1 + aux) 
    return FH_prime


def get_odes(z, Hubble, params_ode, lcdm=False):
    '''
    Returns the system of ODEs for the given cosmological model.


    Parameters:
    -----------
    z : float
        Redshift value.
    variables : list
        List of values for the dynamical variables.
    physical_params : list
        List of model parameters, where the first n-1 elements are the model parameters,
        while the last one specifies the cosmological model. Mathematically, this information is contained in
        the function Gamma.
    model : str, optional
        Cosmological model that is being integrated. Defaults to 'LCDM'.

    Returns:
    -----------
    list
        Set of ODEs for the dynamical variables.
    '''    
    [LAMBDA, L, b, L_bar, H_0, r, s, model] = params_ode #Here L and L_bar are in units of H0^{-1}

    F_H0 = F_H(H_0, [LAMBDA, L, b, L_bar, r, s], model)

    omega_m_0 = F_H0 /(100**2) - OMEGA_R_0

    rho_m_0 = 100**2 * omega_m_0 / KAPPA
    rho_r_0 = 100**2 * OMEGA_R_0 / KAPPA
    
    rho_r = rho_r_0 * (1+z)**4
    rho_m = rho_m_0 * (1+z)**3
    rho_tot =  rho_r + rho_m 
    p_tot =  (1/3) * rho_r

    # To integrate in z
    s =  3 * KAPPA * (rho_tot + p_tot/c_light_km**2) / ((1+z)*F_H_prime(Hubble, [LAMBDA, L, b, L_bar, r, s], model))     
    #print(s)
    return s


def integrator_GILA(physical_params, model, num_z_points=int(10**5),
                initial_z=0, final_z=3,
                system_equations=get_odes, verbose=False,
                method='RK45', rtol=1e-11, atol=1e-16):
 
    t1 = time.time()
    #L_bar, b, H0 = physical_params
    L_bar, b, H0, r, s = physical_params
    
    #Cambiar si pasamos de logspace a linspace!
    zs_int = np.linspace(initial_z, final_z, num_z_points)
    
    #ode_params = [LAMBDA, L/H0, b, L_bar/H0, H0, model]
    ode_params = [LAMBDA, L/H0, b, L_bar/H0, H0, r, s, model]
    sol = solve_ivp(system_equations, (initial_z,final_z),
                    [H0], t_eval=zs_int, args = [ode_params],
                    rtol=rtol, atol=atol, method=method)
        

    assert len(sol.t)==num_z_points, 'Something is wrong with the integration!'
    assert np.all(zs_int==sol.t), 'Not all the values of z coincide with the ones that were required!'

    # Calculate the Hubble parameter
    zs_final = sol.t
    Hs_final = sol.y[0]

    t2 = time.time()

    if verbose == True:
        print('Duration: {} minutes and {} seconds'.format(int((t2-t1)/60),
                int((t2-t1) - 60*int((t2-t1)/60))))

    return zs_final, Hs_final

In [3]:
def chi2_without_cov(teo, data, errors_cuad):
    '''
    Calculate chi square assuming no correlation.

    teo (array): Theoretical prediction of the model.
    data (array): Observational data to compare with the model.
    errors_cuad (array): The square of the errors of the data.

    '''

    chi2 = np.sum((data-teo)**2/errors_cuad)
    return chi2

def params_to_chi2(theta, model, dataset_CC=None,dataset_SN_plus_shoes=None,
                   num_z_points=int(10**5)):

    #chi2_SN = 0
    #chi2_CC = 0
    
    [Mabs, L_bar, b, H_0, r, s] = theta
    physical_params = [L_bar, b, H_0, r, s]
    try:
        zs_model, Hs_model = integrator_GILA(physical_params, model=model, num_z_points=num_z_points,
                                            initial_z=0, final_z=10)
    except Exception as e:
        # If integration fails, reject the step
        return -np.inf

    Hs_interp = interp1d(zs_model, Hs_model)

    int_inv_Hs = cumtrapz(Hs_model**(-1), zs_model, initial=0)
    int_inv_Hs_interp = interp1d(zs_model, int_inv_Hs)

    if dataset_CC != None:
        z_data, H_data, dH = dataset_CC #Import the data
        H_teo = Hs_interp(z_data)
        chi2_CC = chi2_without_cov(H_teo, H_data, dH**2)

    if dataset_SN_plus_shoes != None:
        zhd, zhel, mb, mu_shoes, Cinv, is_cal = dataset_SN_plus_shoes #Import the data
        muobs = mb - Mabs
        muth_num = aparent_magnitude_th(int_inv_Hs_interp, zhd, zhel) #Numeric prediction of mu
        muth = muth_num*(-is_cal + 1) + mu_shoes*(is_cal) #Merge num predicion with mu_shoes
        chi2_SN = chi2_supernovae(muth, muobs, Cinv)


    return chi2_SN + chi2_CC

path_git = git.Repo('.', search_parent_directories=True).working_tree_dir
os.chdir(path_git); os.sys.path.append('./fr_mcmc/utils/')
from data import read_data_chronometers, read_data_pantheon_plus_shoes

# Cosmic Chronometers
os.chdir(path_git+'/fr_mcmc/source/CC/')
ds_CC = read_data_chronometers('chronometers_data.txt')

# Pantheon plus + SH0ES
os.chdir(path_git+'/fr_mcmc/source/Pantheon_plus_shoes')
ds_SN_plus_shoes = read_data_pantheon_plus_shoes('Pantheon+SH0ES.dat',
                                'Pantheon+SH0ES_STAT+SYS.cov')


First, let's fix some parameters

In [4]:
#Fix params
L_bar = 0.90

Gyr_to_second = int(3.1536e16)
Mpc_to_km = int(3.0857e19)
inv_Hub_to_Gyr = Mpc_to_km/Gyr_to_second

z = np.linspace(0, 1000, int(10e3))

num_free_parameters = 3
dof = ds_SN_plus_shoes[2].shape[0]+len(ds_CC[0]) - num_free_parameters

# GILA model analysis

In [5]:
#GILA MODEL
#r_min = 3
#s_min = 1

#r = 3; s = 1
#H0_values = np.linspace(60,80,10)[::-1] 
#beta_values = np.linspace(0.1,12,10)
#Mabs_values = np.linspace(-21,-18.5,10)

#r = 8; s = 1 #Does not work :(
#H0_values = np.linspace(60,80,20)[::-1] 
#beta_values = np.linspace(0.1,12,20)
#Mabs_values = np.linspace(-21,-18.5,20)

#r = 3; s = 2 #It works, not with AoU
#H0_values = np.linspace(60,80,20)[::-1] 
#beta_values = np.linspace(0.1,12,20)
#Mabs_values = np.linspace(-21,-18.5,20)

#r = 3; s = 4 #It works, not with AoU
#H0_values = np.linspace(60,80,20)[::-1] 
#beta_values = np.linspace(0.1,12,20)
#Mabs_values = np.linspace(-21,-18.5,20)

r = 3; s = 5 #It worked, even with AoU!
H0_values = np.linspace(60,80,100)[::-1] 
beta_values = np.linspace(0.1,12,100)
Mabs_values = np.linspace(-21,-18.5,100)

#r = 3; s = 6 #It worked, even with AoU!
#H0_values = np.linspace(60,80,20)[::-1] 
#beta_values = np.linspace(0.1,12,20)
#Mabs_values = np.linspace(-21,-18.5,20)

print(H0_values, beta_values, Mabs_values)

matrix_gila_chi2 = np.zeros((len(H0_values), len(beta_values),len(Mabs_values)))
matrix_gila_aou = np.zeros((len(H0_values), len(beta_values),len(Mabs_values)))
#mask_1_sigma = np.zeros((len(H0_values), len(beta_values),len(Mabs_values)))
#mask_aou = np.zeros((len(H0_values), len(beta_values),len(Mabs_values)))

for (i, (H0_val, beta_val, Mabs_val)) in enumerate(product(H0_values, beta_values, Mabs_values)):
    physical_params = [L_bar, beta_val, H0_val, r, s]
    try :        
        z_final, Hs_final = integrator_GILA(physical_params, model='GILA', num_z_points=int(len(z)),
                                        initial_z=z[0], final_z=z[-1])
    except Exception as e:
        # If integration fails
        matrix_gila_chi2.flat[i] = np.nan
        continue

    theta = [Mabs_val, L_bar, beta_val, H0_val, r, s]
    chi2 = params_to_chi2(theta, dataset_SN_plus_shoes=ds_SN_plus_shoes, dataset_CC=ds_CC, model='GILA') #/ dof
    
    matrix_gila_chi2.flat[i] = chi2 #/ dof
    
    aou_gila = inv_Hub_to_Gyr * simps(((1+z_final) * Hs_final)**(-1), z_final)
    matrix_gila_aou.flat[i] = aou_gila

    #if chi2 <= dof + np.sqrt(2*dof): #change condition with respect to the older notebook
    #    mask_1_sigma.flat[i] = 1

    #if aou_gila >= aou_threshold: #change condition with respect to the older notebook
    #    mask_aou.flat[i] = 1

[80.         79.7979798  79.5959596  79.39393939 79.19191919 78.98989899
 78.78787879 78.58585859 78.38383838 78.18181818 77.97979798 77.77777778
 77.57575758 77.37373737 77.17171717 76.96969697 76.76767677 76.56565657
 76.36363636 76.16161616 75.95959596 75.75757576 75.55555556 75.35353535
 75.15151515 74.94949495 74.74747475 74.54545455 74.34343434 74.14141414
 73.93939394 73.73737374 73.53535354 73.33333333 73.13131313 72.92929293
 72.72727273 72.52525253 72.32323232 72.12121212 71.91919192 71.71717172
 71.51515152 71.31313131 71.11111111 70.90909091 70.70707071 70.50505051
 70.3030303  70.1010101  69.8989899  69.6969697  69.49494949 69.29292929
 69.09090909 68.88888889 68.68686869 68.48484848 68.28282828 68.08080808
 67.87878788 67.67676768 67.47474747 67.27272727 67.07070707 66.86868687
 66.66666667 66.46464646 66.26262626 66.06060606 65.85858586 65.65656566
 65.45454545 65.25252525 65.05050505 64.84848485 64.64646465 64.44444444
 64.24242424 64.04040404 63.83838384 63.63636364 63

In [7]:
np.save(os.path.join(path_data, 'matrix_gila_chi2.npy'), matrix_gila_chi2)
np.save(os.path.join(path_data, 'matrix_gila_aou.npy'), matrix_gila_aou)